In [1]:
!pip install seaborn
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ARDRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import tensorflow as tf
import keras
from keras import backend as K

# Загрузка датасета

Прочитаем датасет

In [3]:
X = pd.read_csv("./X.csv")
Y = pd.read_csv("./y.csv")

In [4]:
df = pd.merge(X, Y, on=["engine_id", "flight_datetime", "flight_phase"])
df.fillna(0, inplace=True)

# Анализ данных

Посмотрим на размер датасета

In [5]:
df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

C:\Users\anchi\AppData\Local\Temp\ipykernel_38836\3129850035.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)
C:\Users\anchi\AppData\Local\Temp\ipykernel_38836\3129850035.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)
C:\Users\anchi\miniconda3\envs\torch\lib\site-packages\pandas\io\formats\style.py:3931: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\anchi\miniconda3\envs\torch\lib\site-packages\pandas\io\formats\style.py:3932: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,engine_position,n1_modifier,number_blades,ZHPTAC,ZLPTAC,ZPCN12,ZPCN25,ZPHSF,ZPHSR,ZPN12R,ZPOIL,ZPS3,ZT1AB,ZT3,ZT49,ZTAMB,ZTLA,ZTNAC,ZTOIL,ZVB1F,ZVB1R,ZVB2F,ZVB2R,ZVSV,ZWF36,IHPSOV,AGW,CAS,IAI,IVS12,SAT,ZALT,ZT1A,ZVIAS,ZWBP1,ZWBP1_8E,ZWBP2,ZWBP2_8E,ZXM,IBE,IBP,IAIE,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,GN2MC,GPCN25,GWFM,PCN12,PCN12I,PCN1AR,PCN1BR,PCN1K,PCN2C,SLOATL,SLOATL_D,VSVNOM,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
engine_position,1.00,-0.00,-0.00,-0.00,-0.00,0.00,0.01,-0.01,0.00,0.00,-0.01,0.00,0.00,0.00,-0.00,-0.00,0.00,0.01,-0.01,-0.04,0.07,0.04,-0.01,-0.01,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.01,-0.00,-0.00,-0.00,-0.01,-0.00,0.01,0.03,0.00,-0.00,0.06,-0.01,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.03,nan,0.00,0.00,-0.00,0.05,-0.03,0.05,0.04,-0.03
n1_modifier,-0.00,1.00,0.67,0.68,0.69,-0.07,0.35,0.69,0.59,-0.60,-0.41,-0.53,0.16,-0.59,-0.53,0.27,-0.59,0.63,0.01,-0.17,0.45,0.56,-0.50,0.54,-0.15,-0.00,0.15,0.71,-0.18,0.02,-0.01,0.26,-0.19,-0.58,0.09,-0.51,0.09,-0.51,0.26,-0.18,-0.76,-0.06,-0.27,-0.51,0.08,0.07,0.62,-0.62,0.18,0.43,0.01,0.67,0.56,0.14,-0.64,-0.25,0.32,0.06,0.06,-0.15,0.26,0.53,0.01,nan,0.09,-0.18,0.39,-0.00,-0.01,0.00,0.02,-0.01
number_blades,-0.00,0.67,1.00,0.82,0.82,-0.25,0.11,0.73,0.70,-0.19,-0.04,-0.17,0.05,-0.19,-0.41,0.08,-0.19,0.80,0.18,-0.02,0.42,0.54,-0.16,0.54,0.15,-0.00,0.10,0.65,-0.06,-0.01,0.03,0.29,-0.24,-0.18,0.15,-0.16,0.15,-0.16,0.29,-0.20,-0.91,-0.10,-0.36,-0.65,-0.21,-0.24,0.74,-0.58,0.26,0.18,-0.00,0.59,0.40,0.35,-0.69,-0.26,0.06,-0.25,-0.25,-0.13,0.30,0.30,-0.00,nan,0.15,-0.54,0.40,-0.00,0.00,0.00,-0.00,-0.00
ZHPTAC,-0.00,0.68,0.82,1.00,0.99,-0.16,0.20,0.68,0.64,-0.45,-0.16,-0.40,0.12,-0.45,-0.62,0.20,-0.45,0.96,0.30,-0.20,0.47,0.54,-0.39,0.65,0.15,-0.00,0.21,0.79,-0.14,0.02,0.02,0.48,-0.41,-0.44,0.16,-0.38,0.16,-0.39,0.49,0.08,-0.84,-0.12,-0.11,-0.68,-0.27,-0.27,0.90,-0.85,0.41,0.30,-0.00,0.85,0.70,0.26,-0.86,-0.25,0.23,-0.29,-0.29,-0.05,0.49,0.40,-0.00,nan,0.16,-0.36,0.64,-0.00,0.00,0.00,-0.00,-0.00
ZLPTAC,-0.00,0.69,0.82,0.99,1.00,-0.19,0.20,0.69,0.65,-0.45,-0.16,-0.40,0.12,-0.45,-0.63,0.20,-0.45,0.96,0.30,-0.20,0.46,0.55,-0.39,0.68,0.15,-0.00,0.19,0.79,-0.14,0.01,0.02,0.48,-0.41,-0.44,0.15,-0.38,0.16,-0.39,0.49,0.08,-0.84,-0.12,-0.11,-0.68,-0.27,-0.27,0.92,-0.85,0.41,0.32,-0.00,0.87,0.71,0.25,-0.86,-0.27,0.22,-0.29,-0.29,-0.08,0.49,0.41,-0.00,nan,0.16,-0.36,0.63,-0.00,-0.00,0.00,-0.00,-0.00
ZPCN12,0.00,-0.07,-0.25,-0.16,-0.19,1.00,0.77,-0.14,-0.13,-0.14,-0.29,-0.14,0.09,-0.13,0.46,0.08,-0.14,-0.17,-0.27,0.07,0.10,-0.00,-0.12,-0.34,-0.21,-0.01,0.03,0.03,-0.06,-0.51,0.06,-0.08,0.17,-0.14,-0.02,-0.15,-0.02,-0.15,-0.06,-0.04,0.22,0.01,0.13,0.13,0.24,0.26,-0.22,0.07,-0.06,0.13,0.00,-0.11,-0.05,-0.11,0.13,0.85,0.30,0.36,0.36,0.65,-0.07,0.13,0.00,nan,-0.02,0.14,-0.05,0.07,0.03,0.00,-0.00,-0.01
ZPCN25,0.01,0.35,0.11,0.20,0.20,0.77,1.00,0.23,0.23,-0.58,-0.59,-0.40,0.41,-0.53,0.11,0.48,-0.53,0.21,-0.38,-0.12,0.42,0.32,-0.47,0.20,-0.26,-0.01,0.13,0.49,-0.10,-0.42,0.10,-0.13,0.26,-0.61,0.04,-0.50,0.04,-0.51,-0.11,-0.15,-0.24,0.01,-0.07,-0.22,0.41,0.41,0.15,-0.27,-0.19,0.50,0.00,0.23,0.24,-0.13,-0.31,0.47,0.29,0.47,0.47,0.28,-0.13,0.56,0.00,nan,0.04,0.05,0.02,0.23,0.07,0.00,0.00,-0.05
ZPHSF,-0.01,0.69,0.73,0.68,0.69,-0.14,0.23,1.00,0.64,-0.40,-0.24,-0.35,0.11,-0.40,-0.44,0.18,-0.40,0.68,0.09,-0.13,0.42,0.49,-0.34,0.51,-0.02,-0.00,0.06,0.63,-0.12,0.01,0.01,0.26,-0.19,-0.39,0.12,-0.34,0.12,-0.34,0.26,-0.15,-0.75,-0.07,-0.26,-0.52,-0.05,-0.07,0.62,-0.57,0.20,0.33,-0.00,0.60,0.47,0.18,-0.61,-0.24,0.20,-0.07,-0.07,-0.14,0.26,0.43,-0.00,nan,0.12,-0.29,0.37,-0.00,-0.01,0.00,0.05,0.00
ZPHSR,0.00,0.59,0.70,0.64,0.65,-0.13,0.23,0.64,1.00,-0.38,-0.24,-0.34,0.10,-0.38,-0.42,0.17,-0.38,0.64,0.07,-0.07,0.40,0.48,-0.32,0.48,-0.03,-0.00,0.09,0.60,-0.12,0.01,0.01,0.24,-0.18,-0.37,0.14,-0.33,0.14,-0.33,0.24,-0.15,-0.71,-0.07,-0.26,-0.49,-0.03,-0.05,0.59,-0.54,0.18,0.31,0.01,0.55,0.43,0.19,-0.58,-0.22,0.19,-

Посмотрим на количество пустых ячеек во входных данных

In [6]:
X.isna().sum()

engine_id              0
aircraft_id            0
flight_datetime        0
flight_phase           0
engine_position        0
n1_modifier            0
number_blades          0
engine_family          0
engine_type            0
manufacturer           0
ZHPTAC             71565
ZLPTAC             71565
ZPCN12                 0
ZPCN25                 0
ZPHSF              68848
ZPHSR              68848
ZPN12R             58916
ZPOIL              19909
ZPS3               58918
ZT1AB              58916
ZT3                58918
ZT49                   0
ZTAMB              58914
ZTLA               58914
ZTNAC              71565
ZTOIL              19909
ZVB1F                 10
ZVB1R              51693
ZVB2F              51718
ZVB2R              15044
ZVSV               34895
ZWF36              19882
IHPSOV             58914
aircraft_family        0
aircraft_type          0
aircraft_grp           0
ac_manufacturer        0
AGW                36674
CAS                51683
IAI                57144


Посмотрим на количество пустых ячеек в выходных данных

In [7]:
Y.isna().sum()

flight_datetime        0
flight_phase           0
engine_id              0
BRAT                 151
DEGT               39069
DELFN              75792
DELN1              75792
DELVSV             71737
DPOIL              57069
EGTC               39069
EGTHDM             21726
EGTHDM_D           57618
GEGTMC             57168
GN2MC              57168
GPCN25             39069
GWFM               39069
PCN12                606
PCN12I             19068
PCN1AR             75741
PCN1BR             75706
PCN1K                815
PCN2C              39069
SLOATL             21726
SLOATL_D           57618
VSVNOM             71737
WBE                68800
WBI                38446
WFMP               39069
ZPCN25_D             282
ZT49_D               282
ZTLA_D             77478
ZTNAC_D            71582
ZWF36_D            20030
dtype: int64

Выделим поля с пропусками

In [8]:
nullCols = ['ZHPTAC', 'ZLPTAC', 'ZPHSF', 'ZPHSR', 'ZPN12R', 'ZPOIL', 'ZPS3', 'ZT1AB', 'ZT3',
       'ZTAMB', 'ZTLA', 'ZTNAC', 'ZTOIL', 'ZVB1F', 'ZVB1R', 'ZVB2F', 'ZVB2R',
       'ZVSV', 'ZWF36', 'IHPSOV', 'AGW', 'CAS', 'IAI', 'IVS12', 'SAT',
       'ZVIAS', 'ZWBP1', 'ZWBP1_8E', 'ZWBP2', 'ZWBP2_8E',
       'IBP', 'IAIE']

Определим категориальные признаки

In [9]:
catCols = ['n1_modifier' ,'flight_phase', 'engine_family', 'engine_type', 'manufacturer', 'aircraft_family', 'aircraft_type', 'aircraft_grp', 'ac_manufacturer']

Подгтовим данные к дальнейшему анализу

In [10]:
def correct_X(X: pd.DataFrame):
    newX = X.drop(columns=["engine_id", "aircraft_id", "flight_datetime"])
    for nullCol in nullCols:
        X.loc[X[X[nullCol].isna()].index, nullCol] = X[nullCol].mean()
    # corrected params
    # theta = newX['ZT1A'] / 288.15
    # P = newX['ZALT'] / 1013.25
    # theta[theta <= 0] = 0.001
    # P[P == 0] = 10
    # newX['EGTK'] = (newX['ZT49'] + 273.15) / theta
    # newX['FFK'] = newX['ZWF36'] / (np.sqrt(theta) * P)
    # newX['N2K'] = newX['ZPCN25'] / np.sqrt(theta)
    # newX['N1K'] = newX['ZPCN12'] / np.sqrt(theta)
    newX = newX.fillna(0)
    # newX = newX.drop(columns=catCols[1:])
    newX = pd.get_dummies(newX, columns=catCols)
    return newX

In [11]:
def correct_Y(Y: pd.DataFrame):
    categorialFeatures = list(Y.columns[Y.dtypes == object])
    newY = Y.drop(columns=categorialFeatures)
    return newY

In [12]:
X = correct_X(X)
y = correct_Y(Y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
X_train.isna().sum()

engine_position                 0
number_blades                   0
ZHPTAC                          0
ZLPTAC                          0
ZPCN12                          0
                               ..
aircraft_grp_B737-BCF           0
aircraft_grp_EMBRAER 170-100    0
ac_manufacturer_AIRBUS          0
ac_manufacturer_BOEING          0
ac_manufacturer_EMBRAER         0
Length: 80, dtype: int64

Отлично, датасет готов

In [14]:
class Trainer:
    def __init__(self, target):
        self.target = target
        self.scaler = StandardScaler()
        self.models = {
            'linear': LinearRegression(),
            'ARD': ARDRegression(),
            'random_tree': RandomForestRegressor(verbose=0),
            'catboost': CatBoostRegressor(iterations=100, verbose=0, random_state=66),
            'XGB': XGBRegressor(random_state=66),
            'nn1': model1(),
            'nn2': model2(),
            'nn3': model3(),
            'nn4': model4(),  
            'nn5': model5(),
            'nn6': model6(),
            'nn7': model7(),
            'nn8': model8()  
        }
    
    def fit(self, X_train, y_train, X_test, y_test):
        X_train_sc = self.scaler.fit_transform(X_train)
        X_test_sc = self.scaler.fit_transform(X_test)
        for name_model in self.models:
            if 'nn' not in name_model:
                self.models[name_model].fit(X_train_sc, y_train[self.target])
            else:
                self.models[name_model].fit(X_train_sc, y_train[self.target], validation_batch_size=50,
                                         validation_data=(X_test_sc, y_test),
                                         epochs=150, verbose=0, callbacks=my_callbacks)
    
    def bestMetric(self):
        return min(self.errors, key=self.errors.get)
    
    def countModelTargetMetric(self, Y):
        e = mean_squared_error

        self.errors, self.metric, self.best_model = {}, {}, {}
        self.metric = {}

        self.errors = {name_model: e(Y[self.target], value, squared=False) for name_model, value in self.predict_data.items()}
        best_model_name = self.bestMetric()
        print(self.target, self.errors)
        self.metric = {
                        'model': best_model_name,
                        'rmse': self.errors[best_model_name]
                      }
        self.best_model = self.models[best_model_name]
        return self.metric
    
    def predictModels(self, X):
        X_scaler = self.scaler.fit_transform(X)
        self.predict_data = { name_model: self.models[name_model].predict(X_scaler) for name_model in self.models} 
        return self.predict_data

    def predict(self, X):
        X_scaler = self.scaler.fit_transform(X)
        predict = pd.DataFrame([])
        predict = self.best_model.predict(X_scaler).flatten()
        return predict

In [15]:
optimizer = keras.optimizers.Adam(0.005, decay=1e-6)

  # Прописываем условия для ранней остановки и управление LR
my_callbacks = [
    #Предполагая, что цель обучения состоит в том, чтобы свести к минимуму потери.
    # Тренировочный цикл будет проверять в конце каждой эпохи: если потери больше не уменьшаются, помечается True, и обучение завершается
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=4),
    # ModelCheckpoint(monitor='val_loss', save_best_only=True, save_weights_only=True),
]

In [16]:
def model1():    
    model = keras.models.Sequential() #группируем слои модели
    # Dense реализует операцию: output = activation(dot(input, kernel) + bias) , где activation — это поэлементная
    # функция активации, переданная в качестве аргумента activation kernel
    model.add(keras.layers.Dense(50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(20, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

model1().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                4050      
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 5,091
Trainable params: 5,091
Non-trainable params: 0
_________________________________________________________________


In [17]:
def model2():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model2().summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                1620      
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,841
Trainable params: 1,841
Non-trainable params: 0
_________________________________________________________________


In [18]:
def model3():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(100, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model3().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               8100      
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 13,201
Trainable params: 13,201
Non-trainable params: 0
_________________________________________________________________


In [19]:
def model4():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(10, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model4().summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 60)                4860      
                                                                 
 dense_10 (Dense)            (None, 30)                1830      
                                                                 
 dense_11 (Dense)            (None, 10)                310       
                                                                 
 dense_12 (Dense)            (None, 1)                 11        
                                                                 
Total params: 7,011
Trainable params: 7,011
Non-trainable params: 0
_________________________________________________________________


In [20]:
def model5():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(50, input_dim=X_train.shape[1], kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(20, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model5().summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 50)                4050      
                                                                 
 dense_14 (Dense)            (None, 20)                1020      
                                                                 
 dense_15 (Dense)            (None, 1)                 21        
                                                                 
Total params: 5,091
Trainable params: 5,091
Non-trainable params: 0
_________________________________________________________________


In [21]:
def model6():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(20, input_dim=X_train.shape[1], kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(10, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model6().summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 20)                1620      
                                                                 
 dense_17 (Dense)            (None, 10)                210       
                                                                 
 dense_18 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1,841
Trainable params: 1,841
Non-trainable params: 0
_________________________________________________________________


In [22]:
def model7():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(100, input_dim=X_train.shape[1], kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(50, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model7().summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 100)               8100      
                                                                 
 dense_20 (Dense)            (None, 50)                5050      
                                                                 
 dense_21 (Dense)            (None, 1)                 51        
                                                                 
Total params: 13,201
Trainable params: 13,201
Non-trainable params: 0
_________________________________________________________________


In [23]:
def model8():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(60, input_dim=X_train.shape[1], kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(30, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(10, kernel_initializer='normal', activation='tanh'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
model8().summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 60)                4860      
                                                                 
 dense_23 (Dense)            (None, 30)                1830      
                                                                 
 dense_24 (Dense)            (None, 10)                310       
                                                                 
 dense_25 (Dense)            (None, 1)                 11        
                                                                 
Total params: 7,011
Trainable params: 7,011
Non-trainable params: 0
_________________________________________________________________


In [24]:
targets = y_train.columns

In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64043 entries, 53491 to 15795
Data columns (total 80 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   engine_position                   64043 non-null  int64  
 1   number_blades                     64043 non-null  float64
 2   ZHPTAC                            64043 non-null  float64
 3   ZLPTAC                            64043 non-null  float64
 4   ZPCN12                            64043 non-null  float64
 5   ZPCN25                            64043 non-null  float64
 6   ZPHSF                             64043 non-null  float64
 7   ZPHSR                             64043 non-null  float64
 8   ZPN12R                            64043 non-null  float64
 9   ZPOIL                             64043 non-null  float64
 10  ZPS3                              64043 non-null  float64
 11  ZT1AB                             64043 non-null  float64
 12  

In [26]:
predict, er = pd.DataFrame([]), pd.DataFrame([])

# Перебираем каждый таргет, стратегия обучения: обучаемся только на тех значениях таргета, где нет пропусков.
# Для этого формируется X_train, y_train для якаждого таргета отдельно
# и на них производится подгонка моделей
for target in tqdm(targets):
    if target == 'VSVNOM':
        predict[target] = 0
        continue
    
  # Обучаем только на непустых данных в таргете
    y_train_ = y_train[y_train[target].notna()]
    X_train_ = X_train[y_train[target].notna()]

    y_test_ = y_test[y_test[target].notna()]
    X_test_ = X_test[y_test[target].notna()]

    # Обучение семейства моделей
    mt = Trainer(target)
    mt.fit(X_train_, y_train_, X_test_, y_test_)

    # Вычисление метрики для каждой модели
    predicts_models = mt.predictModels(X_test_)
    # Лучшая модель и лучшая метрика
    er[target] = mt.countModelTargetMetric(y_test_)

    # На лучшей модели предсказываем тестовую выборку
    predict[target] = mt.predict(X_test)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

985/985 [==============================] - 0s 402us/step
BRAT {'linear': 665986528.2575403, 'ARD': 0.04944316612520106, 'random_tree': 0.017731056502284112, 'catboost': 0.0174844850846705, 'XGB': 0.017456989584479693, 'nn1': 0.03363080865509308, 'nn2': 0.029034353744289542, 'nn3': 0.19924703182806117, 'nn4': 0.03905914455753735, 'nn5': 0.030479127498219672, 'nn6': 0.029444731485534285, 'nn7': 0.038224936321524794, 'nn8': 0.03213929932433506}


  3%|██▊                                                                                | 1/30 [01:10<34:17, 70.96s/it]

582/582 [==============================] - 0s 404us/step
DEGT {'linear': 40719055639.61503, 'ARD': 3.5144413124834495, 'random_tree': 4.3453549408211805, 'catboost': 3.5858587117971554, 'XGB': 3.6516753121438197, 'nn1': 1.6683593932505687, 'nn2': 3.0222154077880887, 'nn3': 2.3997966523090613, 'nn4': 1.8410815208944797, 'nn5': 12.501418579023685, 'nn6': 7.117638310398669, 'nn7': 12.506789494288592, 'nn8': 12.505720588634071}
986/986 [==============================] - 0s 376us/step


  7%|█████▌                                                                             | 2/30 [02:24<33:54, 72.65s/it]

...

 93%|████████████████████████████████████████████████████████████████████████████▌     | 28/30 [30:15<02:49, 84.83s/it]

249/249 [==============================] - 0s 371us/step
ZTNAC_D {'linear': 2602588891422.495, 'ARD': 5.405054964668252, 'random_tree': 2.344349042741341, 'catboost': 2.518667155313914, 'XGB': 2.4290549394141876, 'nn1': 2.448744861341816, 'nn2': 2.6462523743016044, 'nn3': 2.2494334364965916, 'nn4': 2.3772091306948155, 'nn5': 2.176944500930314, 'nn6': 2.5031738117258904, 'nn7': 2.1575196498515403, 'nn8': 2.3037056843459607}
986/986 [==============================] - 0s 363us/step


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 29/30 [30:46<01:09, 69.37s/it]

777/777 [==============================] - 0s 371us/step
ZWF36_D {'linear': 722085510648.2996, 'ARD': 102.89661462258474, 'random_tree': 122.64800428608297, 'catboost': 116.51992268174318, 'XGB': 124.67332630982922, 'nn1': 115.80127950115218, 'nn2': 137.82113668782947, 'nn3': 102.26344314875699, 'nn4': 99.13894556437762, 'nn5': 140.29859499441997, 'nn6': 141.023764300433, 'nn7': 139.27839212380687, 'nn8': 142.38362625756193}
986/986 [==============================] - 0s 368us/step


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [33:11<00:00, 66.37s/it]


In [27]:
er

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,...,SLOATL,SLOATL_D,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
model,XGB,nn1,nn3,nn3,ARD,random_tree,catboost,nn3,XGB,nn3,...,nn4,nn7,random_tree,XGB,XGB,random_tree,random_tree,random_tree,nn7,nn4
rmse,0.017457,1.668359,0.457241,0.246441,0.009416,0.105439,2.109889,3.698539,5.229893,2.181329,...,1.194924,1.517889,0.350584,0.018939,21.176542,0.488191,7.473767,0.056401,2.15752,99.138946


In [28]:
er.iloc[0]

BRAT                XGB
DEGT                nn1
DELFN               nn3
DELN1               nn3
DELVSV              ARD
DPOIL       random_tree
EGTC           catboost
EGTHDM              nn3
EGTHDM_D            XGB
GEGTMC              nn3
GN2MC               nn1
GPCN25              XGB
GWFM                XGB
PCN12       random_tree
PCN12I              ARD
PCN1AR              ARD
PCN1BR         catboost
PCN1K               ARD
PCN2C               XGB
SLOATL              nn4
SLOATL_D            nn7
WBE         random_tree
WBI                 XGB
WFMP                XGB
ZPCN25_D    random_tree
ZT49_D      random_tree
ZTLA_D      random_tree
ZTNAC_D             nn7
ZWF36_D             nn4
Name: model, dtype: object

In [29]:
er.iloc[1]

BRAT         0.017457
DEGT         1.668359
DELFN        0.457241
DELN1        0.246441
DELVSV       0.009416
DPOIL        0.105439
EGTC         2.109889
EGTHDM       3.698539
EGTHDM_D     5.229893
GEGTMC       2.181329
GN2MC        0.083425
GPCN25       0.102216
GWFM         0.725192
PCN12        0.073542
PCN12I       0.033699
PCN1AR       0.112558
PCN1BR       2.660083
PCN1K        0.127223
PCN2C        0.092423
SLOATL       1.194924
SLOATL_D     1.517889
WBE          0.350584
WBI          0.018939
WFMP        21.176542
ZPCN25_D     0.488191
ZT49_D       7.473767
ZTLA_D       0.056401
ZTNAC_D       2.15752
ZWF36_D     99.138946
Name: rmse, dtype: object

In [30]:
predict

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,...,SLOATL_D,VSVNOM,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
0,0.981547,28.698778,29.173315,12.541180,2.122546,-20.596460,641.140476,24.248169,-2.525459,83.768059,...,-1.791492,0,0.726202,1.000018,2587.232178,0.001898,6.802498,-0.006403,-8.495239,8.649666
1,1.001632,44.173973,53.290943,21.807476,2.122546,-23.513668,645.283348,73.572365,-1.903859,45.623482,...,-1.122995,0,0.717383,0.920597,1800.514404,-0.033450,-3.375000,0.031404,-11.235924,-35.548054
2,0.170122,21.239252,20.651855,10.022750,2.122546,-28.092729,716.005095,7.922024,-4.496054,64.655014,...,-1.930697,0,1.464971,0.011049,1842.516235,-0.138499,4.935000,0.053592,-6.694004,-76.818794
3,1.001669,35.305969,54.328987,22.254789,2.122546,-23.193008,613.433559,81.945015,-0.462623,48.516544,...,-1.178614,0,0.710108,0.918797,1783.883423,-0.040950,-0.525000,0.014844,-7.956555,-35.332695
4,0.992322,4.840224,37.992710,15.226684,2.122546,-18.974609,595.696772,58.442318,-1.606995,86.651482,...,-0.478632,0,0.710328,0.999983,2520.940430,-0.054203,1.992499,-0.002165,15.336040,-7.995853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31540,0.455855,-35.001553,14.255494,5.648674,3.539746,-21.034884,613.760473,71.078720,-2.809794,201.132355,...,-2.297027,0,1.386045,0.126157,2629.913818,0.028000,-0.364500,0.005492,8.101039,49.141052
31541,1.001592,43.799419,56.966442,24.337326,2.122546,-23.688507,658.475464,47.985916,7.550748,42.818962,...,2.332056,0,0.718706,0.928811,1813.258789,0.029849,-12.905000,0.024997,-2.794729,-97.297562
31542,0.285449,-8.517652,15.924258,8.633173,2.395085,-20.834144,638.659560,7.487614,-2.741263,122.317612,...,0.922978,0,1.468498,-0.002468,2899.909424,0.138501,-3.903002,-0.003885,18.104425,56.940750
31543,0.992292,-31.863485,0.345617,0.216551,1.304931,-13.339090,676.747432,49.446857,6.052213,0.661275,...,-1.183674,0,0.705037,-0.000090,1973.125854,-0.036876,-6.460000,-0.025003,-18.921463,-16.781221


In [31]:
y_test

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,...,SLOATL_D,VSVNOM,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
22023,1.000000,22.495361,NaN,NaN,NaN,-16.489998,686.504639,56.252117,NaN,61.504601,...,NaN,NaN,NaN,1.000000,2960.08154,0.000000,5.500000,NaN,NaN,1.899902
74546,1.000000,NaN,25.345905,10.062073,NaN,NaN,NaN,71.308876,5.089859,NaN,...,1.696529,NaN,NaN,NaN,NaN,-0.040001,-5.000000,NaN,NaN,NaN
63711,0.777497,NaN,22.256674,9.457748,NaN,NaN,NaN,19.559158,-4.640778,NaN,...,-1.364059,NaN,1.23459,NaN,NaN,-0.099998,5.000000,NaN,NaN,NaN
78787,1.000000,NaN,25.713123,9.781273,NaN,NaN,NaN,67.841065,2.325699,NaN,...,0.774151,NaN,NaN,NaN,NaN,0.040001,-1.000000,NaN,NaN,NaN
85802,1.000000,19.326294,NaN,NaN,NaN,-9.752174,609.773682,57.439178,NaN,64.673698,...,NaN,NaN,NaN,1.000000,2653.28687,-0.029999,2.049988,NaN,NaN,5.899902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51004,0.893764,-14.823608,NaN,NaN,2.6,-23.015934,617.423584,48.106640,NaN,128.823593,...,NaN,0.0,0.92594,-0.106236,2673.01562,0.050003,-1.349976,NaN,4.0,1.102295
23839,1.000000,NaN,29.984722,13.078873,NaN,NaN,NaN,51.658375,14.686249,NaN,...,4.635387,NaN,NaN,NaN,NaN,0.059998,-13.500000,NaN,NaN,NaN
11940,0.851788,-5.938904,NaN,NaN,0.5,-20.552170,664.138916,14.728130,NaN,62.938900,...,NaN,0.0,1.12436,-0.148212,3435.62183,0.149994,-3.549988,NaN,-0.5,14.330078
23151,1.000000,15.805420,NaN,NaN,NaN,NaN,725.819580,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,1711.99304,-0.078102,-5.875000,-0.0625,NaN,-44.560059
